In [1]:
import pandas as pd
# from sklearn.feature_extraction.text import CountVectorizer

full = pd.read_csv("../../data/poptrag_lyrics_genres_corpus_filtered_english_lemmatized.csv")
full.info()

# top 20 most common words in the lyrics
# def print_most_common_words(corpus, lyrics_column, top_n=20):
#     vectorizer = CountVectorizer(
#                 ngram_range=(1, 1),
#                 token_pattern=r"\b[\w']+\b",
#                 lowercase=True,
#             )
#     matrix = vectorizer.fit_transform(full[lyrics_column])
#     sum_words = matrix.sum(axis=0)
#     words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
#     words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
#     print(words_freq[:20])

# print_most_common_words(full, "full_lyrics")
# print("=" * 60)
# print_most_common_words(full, "lyrics_lemmatized")

<class 'pandas.DataFrame'>
RangeIndex: 111938 entries, 0 to 111937
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                111938 non-null  int64  
 1   track.s.id                111938 non-null  str    
 2   track.s.title             111937 non-null  str    
 3   track.s.firstartist.name  111938 non-null  str    
 4   album.s.title             111938 non-null  str    
 5   album.s.releaseyear       111938 non-null  int64  
 6   track.s.popularity        111938 non-null  int64  
 7   track.language            111938 non-null  str    
 8   full_lyrics               111938 non-null  str    
 9   cat5                      111938 non-null  str    
 10  pmax5                     111938 non-null  float64
 11  nmax5                     111938 non-null  float64
 12  cat12                     111938 non-null  str    
 13  pmax12                    111938 non-null  float64
 14 

In [71]:
from helpers.LyricsClassficationExperiment import LyricsClassificationExperiment

exp_fs = LyricsClassificationExperiment(
    corpus=full, 
    genrecol="cat12",
    lyricscol="full_lyrics", 
    artistcol="track.s.firstartist.name", 
    output_dir="cat5_mock_experiment_fs",
    test_size=0.2,
    random_state=42, 
    subsample_debug=0.05,
)
exp_fs.compute_fs_ngram_features(min_artists=20, top_n_per_genre_and_ngram=100)
fs_features = exp_fs.X_train.keys()
print(exp_fs)
print(type(exp_fs.X_train))
exp_fs.train_fixed_parametrer_logistic_regression()
exp_fs.show_model_evaluation()
exp_fs.show_top_coefficients_per_genre()

Extracted unigrams:
  - Unique: 21,575
  - Shape: (4458, 21575)
  - Examples: ['wives', 'colt', 'antibiotics', 'huckleberry', 'grams']
Extracted bigrams:
  - Unique: 194,350
  - Shape: (4458, 194350)
  - Examples: ['to department', "castaway i'll", 'all wanting', 'her lively', 'going unconditional']
Extracted trigrams:
  - Unique: 410,566
  - Shape: (4458, 410566)
  - Examples: ["things you've always", 'calling save me', 'air to soothe', 'working on the', 'hotter in your']
Calculating genre-level TF-IDF for unigrams with genre ...
Calculated TF-IDF for 51,624 genre-ngram pairs
Calculating genre-level TF-IDF for bigrams with genre ...
Calculated TF-IDF for 285,738 genre-ngram pairs
Calculating genre-level TF-IDF for trigrams with genre ...
Calculated TF-IDF for 467,138 genre-ngram pairs
Counting artists per n-gram...
  10% complete
  20% complete
  30% complete
  40% complete
  50% complete
  60% complete
  70% complete
  80% complete
  90% complete
Calculated artist diversity for 21,57

N:\Materialien\Promotion\LyricsGenreRecognition\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
from helpers.LyricsClassficationExperiment import LyricsClassificationExperiment

exp_informed = LyricsClassificationExperiment(
    corpus=full,
    genrecol="cat12",
    lyricscol="lyrics_lemmatized",
    artistcol="track.s.firstartist.name",
    output_dir="cat5_mock_experiment_informed",
    test_size=0.2,
    random_state=42,
    subsample_debug=0.05,
)
exp_informed.compute_idiom_ngram_features(min_artists=20, llr_treshold=10, top_n_per_genre=300) # to be similar to FS (2014)
print(exp_informed)
print(type(exp_informed.X_train))
exp_informed.train_fixed_parametrer_logistic_regression()
exp_informed.show_model_evaluation()

Step 1: Extracting and scoring n-grams per genre...
Extracted 167705 unique bigrams, 380098 unique trigrams
Filtered to 2006 n-grams (>= 20 artists, no stopwords)
Filtered to 417 n-grams (>= 20 artists, no stopwords)
Selected 311 bigrams and 227 trigrams across genres

Step 2: Replacing n-grams in corpus...

Step 3: Extracting unigrams from replaced corpus...
Extracted 1322 unigrams meeting criteria

Step 4: Ranking all tokens via TF-IDF...

Final vocabulary size: 1216
LyricsClassificationExperiment with 11 genres
Train size: 4458 samples
Test size: 1138 samples
# of features: 1216
Feature type: Informed N-grams (top 300 per genre, min. 20 artists)
Model not yet trained.
Output directory: cat5_mock_experiment_informed

<class 'pandas.DataFrame'>
Training pipeline with fixed parameters...
Selected model parameters:
  C: 1.000
  l1_ratio: 0.500
  target_ratio: 3.000
F1 macro: 0.212
Precision macro: 0.222
Recall macro: 0.222
Cohen's kappa: 0.158
                  precision    recall  f1-s

N:\Materialien\Promotion\LyricsGenreRecognition\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [75]:
exp_informed.show_top_coefficients_per_genre()

Top 10 coefficients for genre: ALTERNATIVE ROCK
hand (0.458)
grace (0.426)
distance (0.410)
metal (0.407)
pretend (0.394)
conquer (0.380)
world (-0.377)
root (0.358)
lightning (0.355)
moment (-0.346)


Top 10 coefficients for genre: ELECTRONIC
your_way (-0.623)
man (-0.568)
clear (-0.547)
believe (-0.518)
you_get_to (0.503)
you_walk (0.503)
family (0.496)
cat (0.493)
day (0.480)
golden (-0.479)


Top 10 coefficients for genre: HARD ROCK
madness (0.464)
thrill (0.432)
that_make (0.412)
spot (0.363)
leave (0.357)
appear (0.353)
the_key (0.343)
fade (0.335)
wheel (0.334)
the_song (0.329)


Top 10 coefficients for genre: HEAVY METAL
oh_baby (0.560)
gonna (0.451)
in_life (0.420)
the_fear (0.394)
attack (0.383)
spell (0.364)
no_time_to (0.355)
fail (0.349)
i_keep (-0.332)
luck (0.331)


Top 10 coefficients for genre: HIP HOP
nigga (0.581)
true (0.426)
em (0.419)
i_get_a (0.408)
rap (0.406)
shit (0.399)
man (0.375)
money (0.362)
dollar (0.362)
let_me (0.357)


Top 10 coefficients for genre: I

In [67]:
from helpers.LyricsClassficationExperiment import LyricsClassificationExperiment

exp_topics = LyricsClassificationExperiment(
    corpus=full,
    genrecol="cat12",
    lyricscol="lyrics_lemmatized",
    artistcol="track.s.firstartist.name",
    output_dir="cat5_mock_experiment_informed",
    test_size=0.2,
    random_state=42,
    subsample_debug=0.05,
)
exp_topics.compute_idiom_ngram_features(min_artists=50, llr_threshold=10, top_n_per_genre=300)
exp_topics.corpus_train_replaced.to_csv("corpus_train_replaced_toR.csv", index=False)
exp_topics.corpus_test_replaced.to_csv("corpus_test_replaced_toR.csv", index=False)

Step 1: Extracting and scoring n-grams per genre...
Extracted 167705 unique bigrams, 380098 unique trigrams
Filtered to 548 n-grams (>= 50 artists, no stopwords)
Filtered to 51 n-grams (>= 50 artists, no stopwords)
Selected 168 bigrams and 39 trigrams across genres

Step 2: Replacing n-grams in corpus...

Step 3: Extracting unigrams from replaced corpus...
Extracted 646 unigrams meeting criteria

Step 4: Ranking all tokens via TF-IDF...

Final vocabulary size: 837


In [49]:
# BTM features
exp_topics.compute_topics_features(n_topics = (20, 20, 1))

100%|██████████| 200/200 [06:28<00:00,  1.94s/it]


In [50]:
print("Vocabulary of Topic Model:")
print(exp_topics.topic_model.vocabulary_)
print("Topic Words:")
exp_topics.topic_model.get_topic_words()


Vocabulary of Topic Model:
['a_little' 'a_long' 'a_man' 'a_new' 'a_thousand' 'a_time' 'a_world' 'across_the' 'act' 'afraid' 'ah' 'ahead' 'alive' 'alone' 'along' 'alright' 'always' 'and_get' 'and_i_know' 'and_you_know' 'angel' 'another' 'apart' 'arm' 'around'
 'ask' 'away' 'babe' 'baby' 'baby_i' 'baby_you' 'back' 'back_to' 'bad' 'bag' 'be_bear' 'be_come' 'be_go' 'be_mine' 'be_right' 'be_wait' 'bear' 'beat' 'beautiful' 'become' 'bed' 'begin' 'behind' 'believe' 'belong' 'best' 'beyond' 'big'
 'bird' 'bitch' 'black' 'bleed' 'blind' 'blood' 'blow' 'blue' 'body' 'bone' 'book' 'bout' 'boy' 'brain' 'break' 'break_the' 'breath' 'breathe' 'bright' 'bring' 'bring_me' 'broken' 'brother' 'build' 'burn' 'burning' 'buy' 'call'
 'call_me' 'can_feel' 'can_take' 'cannot' 'car' 'care' 'carry' 'cast' 'catch' 'cause' 'cause_i' 'cause_you' 'chain' 'chance' 'change' 'check' 'child' 'choose' 'city' 'clean' 'clear' 'close' 'clothes' 'cloud' 'cold' 'come' 'come_on'
 'come_to' 'control' 'cool' 'cover' 'crash' 'c

{0: ['keep',
  'well',
  'u',
  'try_to',
  'nothing',
  'your_heart',
  'gonna',
  'away',
  'roll',
  'deep'],
 1: ['nigga',
  'fuck',
  'shit',
  'bitch',
  'like',
  'yo',
  'get',
  'em',
  'i_get',
  'yeah'],
 2: ['find',
  'ever',
  'forever',
  'inside',
  'run',
  'kill',
  'life',
  'always',
  'stop',
  'die'],
 3: ['time',
  'keep',
  'day',
  'since',
  'like',
  'remember',
  'bad',
  'live',
  'ever',
  'never'],
 4: ['king',
  'come',
  'u',
  'find',
  'maybe',
  'never',
  'go',
  'dream',
  'be_come',
  'thing'],
 5: ['bout',
  'talkin',
  'worry',
  'life',
  'people',
  'get',
  'think',
  'stop',
  'shit',
  'money'],
 6: ['fight',
  'go',
  'right',
  'another',
  'cause',
  'last',
  'one',
  'get',
  'back',
  'chance'],
 7: ['get',
  'back',
  'train',
  'go',
  'one',
  'inside',
  'girl',
  'wait',
  'home',
  'lose'],
 8: ['i_love_you',
  'love_you',
  'word',
  'remember',
  'my_heart',
  'yeah_yeah',
  'my_name',
  'uh',
  'say_i',
  'way_to'],
 9: ['love

In [62]:
# exp_topics.topics_extractor.get_topic_score(exp_topics.corpus_train_replaced)
exp_topics.topic_model.coherence_

ValueError: Buffer dtype mismatch, expected 'long' but got 'long long'

In [52]:
exp_topics.train_fixed_parametrer_logistic_regression()
exp_topics.show_model_evaluation()

Training pipeline with fixed parameters...
Selected model parameters:
  C: 1.000
  l1_ratio: 0.500
  target_ratio: 3.000
F1 macro: 0.222
Precision macro: 0.235
Recall macro: 0.235
Cohen's kappa: 0.223
                  precision    recall  f1-score   support

alternative rock       0.18      0.07      0.10        46
      electronic       0.19      0.11      0.14        90
       hard rock       0.00      0.00      0.00        40
     heavy metal       0.12      0.07      0.09        56
         hip hop       0.70      0.61      0.65        76
      indie rock       0.11      0.03      0.05        31
            jazz       0.00      0.00      0.00        22
           metal       0.48      0.58      0.53       110
             pop       0.36      0.55      0.43       239
        pop rock       0.00      0.00      0.00        16
            rock       0.45      0.47      0.46       412

        accuracy                           0.40      1138
       macro avg       0.24      0.23      

N:\Materialien\Promotion\LyricsGenreRecognition\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [73]:
# perform STM in R and load and set X_train and X_test
X_train = pd.read_csv('X_train_stm.csv')
X_test = pd.read_csv("X_test_stm.csv")

# bad practice, but ok for now
exp_topics.X_train = X_train
exp_topics.X_test = X_test

exp_topics.train_fixed_parametrer_logistic_regression()


Training pipeline with fixed parameters...


N:\Materialien\Promotion\LyricsGenreRecognition\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [74]:
exp_topics.show_model_evaluation()

Selected model parameters:
  C: 1.000
  l1_ratio: 0.500
  target_ratio: 3.000
F1 macro: 0.184
Precision macro: 0.201
Recall macro: 0.201
Cohen's kappa: 0.145
                  precision    recall  f1-score   support

alternative rock       0.07      0.07      0.07        46
      electronic       0.09      0.10      0.09        90
       hard rock       0.06      0.05      0.05        40
     heavy metal       0.06      0.07      0.07        56
         hip hop       0.77      0.43      0.55        76
      indie rock       0.00      0.00      0.00        31
            jazz       0.00      0.00      0.00        22
           metal       0.39      0.37      0.38       110
             pop       0.35      0.45      0.39       239
        pop rock       0.00      0.00      0.00        16
            rock       0.43      0.40      0.41       412

        accuracy                           0.32      1138
       macro avg       0.20      0.18      0.18      1138
    weighted avg       0.33 

In [10]:
from helpers.LyricsClassficationExperiment import LyricsClassificationExperiment

exp_topics = LyricsClassificationExperiment(
    corpus=full,
    genrecol="cat12",
    lyricscol="lyrics_lemmatized",
    artistcol="track.s.firstartist.name",
    output_dir="cat5_mock_experiment_informed",
    test_size=0.2,
    random_state=42,
    subsample_debug=0.05,
)
exp_topics.compute_idiom_ngram_features(
    min_artists=20, min_tracks=40, llr_threshold=10, top_n_per_ngram_pergenre=100,
)

Extracting and scoring n-grams per genre...
Extracted 167705 unique bigrams, 380098 unique trigrams, 480992 unique quadgrams
Filtered to 639 n-grams (>= 20 artists, >= 40 tracks, no stopwords)
Filtered to 91 n-grams (>= 20 artists, >= 40 tracks, no stopwords)
Filtered to 2 n-grams (>= 20 artists, >= 40 tracks, no stopwords)
Selected 585 bigrams, 91 trigrams, and 2 quadgrams across genres

Replacing n-grams in corpus...

Ranking tokens via TF-IDF per n-gram type...

Final vocabulary size: 487


In [12]:
# exp_topics.ngram_extractor.vocabulary

# filter quadgrams (containing three underscores) from vocab
filtered_vocab = [
    token for token in exp_topics.ngram_extractor.vocabulary
    if token.count('_') >= 2
]
print(f"Filtered tokens (>=3 underscores): {len(filtered_vocab)}")
filtered_vocab

Filtered tokens (>=3 underscores): 93


['a_little_bit',
 'a_lot_of',
 'all_the_thing',
 'all_the_time',
 'and_i_know',
 'and_you_know',
 'as_long_a',
 'back_to_the',
 'be_the_one',
 'but_i_know',
 'close_your_eye',
 'find_a_way',
 'i_can_feel',
 'i_can_see',
 'i_feel_the',
 'i_get_a',
 'i_get_the',
 'i_get_to',
 'i_give_you',
 'i_just_wanna',
 'i_know_i',
 'i_know_it',
 'i_know_that',
 'i_know_what',
 'i_know_you',
 'i_love_you',
 'i_need_a',
 'i_need_to',
 'i_need_you',
 'i_see_the',
 'i_see_you',
 'i_tell_you',
 'i_think_i',
 'i_try_to',
 'i_use_to',
 'i_wanna_be',
 'i_want_to',
 'i_want_you',
 'if_you_want',
 'in_love_with',
 'in_my_eye',
 'in_my_head',
 'in_my_heart',
 'in_my_mind',
 'in_the_air',
 'in_the_dark',
 'in_the_end',
 'in_the_morning',
 'in_the_night',
 'in_the_sky',
 'in_your_eye',
 'know_how_to',
 'know_that_i',
 'know_that_you',
 'know_what_i',
 'look_at_me',
 'my_heart_be',
 'of_the_night',
 'of_the_world',
 'oh_oh_oh',
 'oh_oh_oh_oh',
 'on_my_mind',
 'tell_me_that',
 'tell_me_what',
 'the_end_of',
 'the_

In [13]:
exp_topics.train_fixed_parametrer_logistic_regression()
exp_topics.show_model_evaluation()

Training pipeline with fixed parameters...
Selected model parameters:
  C: 1.000
  l1_ratio: 0.500
  target_ratio: 3.000
F1 macro: 0.140
Precision macro: 0.139
Recall macro: 0.139
Cohen's kappa: 0.073
                  precision    recall  f1-score   support

alternative rock       0.08      0.09      0.09        46
      electronic       0.12      0.13      0.12        90
       hard rock       0.04      0.03      0.03        40
     heavy metal       0.09      0.11      0.10        56
         hip hop       0.27      0.36      0.31        76
      indie rock       0.02      0.03      0.03        31
            jazz       0.03      0.05      0.04        22
           metal       0.20      0.19      0.19       110
             pop       0.30      0.34      0.32       239
        pop rock       0.00      0.00      0.00        16
            rock       0.38      0.28      0.32       412

        accuracy                           0.24      1138
       macro avg       0.14      0.14      

N:\Materialien\Promotion\LyricsGenreRecognition\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [16]:
exp_topics.show_random_baseline_evaluation()

F1 macro: 0.086
Precision macro: 0.087
Recall macro: 0.087
Cohen's kappa: -0.001
